 # Predicting Book Success

In [1]:
#Import Dependencies
from path import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf



In [2]:

#Import the dataset
data = Path('books_clean.csv')
df = pd.read_csv(data)

# We can see from the preview of the DataFrame that multiple variables (also called features), such as the isbn13,
#published_year, average_rating, num_pages, ratings_count, can be used to predict the outcome: whether a book will have a good 
#rating (1) or will not (0) based on the fact that an average rating below of 4.5 will not likely be successful

df.loc[df['Rating'] <= 4.1, 'Rating_Classification'] = 'Low_Rating' 
df.loc[df['Rating'] > 4.1, 'Rating_Classification'] = 'High_Rating'

df.reset_index(inplace=True, drop=True)
df.head()

,ISBN,Rating,Price,Pages,Rating Count,Language_Dummy,Size_Dummy,Famous_Dummy,Categories_Dummy,Serie_Dummy,Rating_Classification
0,9.780000e+12,4.49,7.38,870,1996446,English,Big,Famous,Top_Category,Serie,High_Rating
1,9.780000e+12,3.59,2.10,498,4367341,English,Big,Other,Other,Serie,Low_Rating
2,9.780000e+12,4.59,21.15,1728,97731,English,Big,Famous,Top_Category,Serie,High_Rating
3,9.780000e+12,3.84,2.73,464,1041597,English,Big,Other,Top_Category,Other,Low_Rating
4,9.780000e+12,4.11,5.46,532,1328143,English,Big,Other,Top_Category,Other,High_Rating


In [3]:
# We will check for the variables from all columns
df.dtypes

ISBN                     float64
Rating                   float64
Price                    float64
Pages                      int64
Rating Count               int64
Language_Dummy            object
Size_Dummy                object
Famous_Dummy              object
Categories_Dummy          object
Serie_Dummy               object
Rating_Classification     object
dtype: object

In [4]:
#Count the current NaN values from the dataframe
df.isnull().sum().sum()

0

In [5]:
#Drop all the NaN values
df.dropna(inplace=True)

In [6]:
#Recount all the NaN values to make sure they are dropped
df.isnull().sum().sum()

0

In [7]:
#Use method to convert String to int
def rating(x):
    if x == 'Low_Rating':
        return 0
    if x == 'High_Rating':
        return 1
    

In [8]:
#Apply the method to the rating_classification column
df['Rating_Classification'] = df['Rating_Classification'].apply(rating)

In [9]:
#Use method to convert String to int
def binary(x):
    if x == 'Other':
        return 0
    if x == 'English'or 'Big' or 'Famous' or 'Serie' or 'Top_Category':
        return 1

In [10]:
#Apply the method to the rating_classification column
df['Size_Dummy'] = df['Size_Dummy'].apply(binary)

In [11]:
df['Serie_Dummy'] = df['Serie_Dummy'].apply(binary)

In [12]:
df['Famous_Dummy'] = df['Famous_Dummy'].apply(binary)

In [13]:
df['Language_Dummy'] = df['Language_Dummy'].apply(binary)

In [14]:
df['Categories_Dummy'] = df['Categories_Dummy'].apply(binary)

In [15]:
#Create a new dataframe with the target variables
df.tail(50)

,ISBN,Rating,Price,Pages,Rating Count,Language_Dummy,Size_Dummy,Famous_Dummy,Categories_Dummy,Serie_Dummy,Rating_Classification
1871,9.780000e+12,4.08,4.99,294,5255,1,0,1,0,0,0
1872,9.780000e+12,3.85,3.97,150,1357,1,1,1,1,0,0
1873,9.780000e+12,4.01,4.54,240,2714,1,1,0,0,0,0
1874,9.780000e+12,4.22,7.73,192,2687,1,0,0,1,0,1
1875,9.780000e+12,4.20,4.14,502,2326,1,1,0,0,0,1
1876,9.780000e+12,4.07,1.92,452,21141,1,0,1,1,0,0
1877,9.780000e+12,4.09,4.16,368,3663,1,0,0,1,1,0
1878,9.780000e+12,3.36,1.92,272,15787,1,1,1,1,1,0
1879,9.780000e+12,3.89,2.86,384,3607,1,1,1,1,0,0
1880,9.780000e+12,4.10,5.28,352,3479,1,1,0,1,1,0


##  Separate the Features (X) from the Target (y)

In [16]:
#The Outcome column is defined as y, or the target.
#X, or features, is created by dropping the Outcome column from the DataFrame.

y = df["Rating_Classification"]
X = df.drop(columns=["Rating_Classification","ISBN","Rating","Rating Count"])



 ## Split our data into training and testing

In [17]:
#We first split the dataset into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape


(1440, 7)

In [18]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [19]:
#Examining the shape of the training set with X_train.shape returned (1636,5), meaning that there are 1636 samples (rows) and 
#five features (columns).

In [20]:
#The next step was to create a logistic regression model with the specified arguments for solver, max_iter, and random_state
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs',
                                max_iter=200,
                                random_state=1)

In [21]:
#we trained the model with the training data
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=200, random_state=1)

In [22]:
#To create predictions for y-values, we used the X_test set
y_pred = classifier.predict(X_test)
results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test}).reset_index(drop=True)
results.tail(20)


,Prediction,Actual
461,0,1
462,0,0
463,0,1
464,0,0
465,0,1
466,0,0
467,0,0
468,0,0
469,0,0
470,0,0


In [23]:
#The final step is to measure the accuracy of the logistic regression model created

from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.7068607068607069


In [24]:
#taking into account that the accuracy score is simply the percentage of predictions that are correct. In this case, 
#the model's accuracy score was 0.9834, meaning that the model was correct 98.34% of the time.

In [25]:
#  import the relevant modules for validation and print the confusion_matrix
from sklearn.metrics import confusion_matrix, classification_report
matrix = confusion_matrix(y_test, y_pred)
matrix_df = pd.DataFrame(
    matrix, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
matrix_df

,Predicted 0,Predicted 1
Actual 0,314,14
Actual 1,127,26


In [26]:
# Report of sensitivity, precission and F1. La línea 0 se aplica al predictor de diabetes como hipótesis alternativa
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.71      0.96      0.82       328
           1       0.65      0.17      0.27       153

    accuracy                           0.71       481
   macro avg       0.68      0.56      0.54       481
weighted avg       0.69      0.71      0.64       481



# Single layer

In [27]:
# Define the basic neural network model
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=1, activation="relu", input_dim=len(X.columns)))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=200)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/200
45/45 [==============================] - 2s 4ms/step - loss: 0.6512 - accuracy: 0.6819
Epoch 2/200
45/45 [==============================] - 0s 3ms/step - loss: 0.6461 - accuracy: 0.6819
Epoch 3/200
45/45 [==============================] - 0s 5ms/step - loss: 0.6420 - accuracy: 0.6819
Epoch 4/200
45/45 [==============================] - 0s 4ms/step - loss: 0.6384 - accuracy: 0.6819
Epoch 5/200
45/45 [==============================] - 0s 3ms/step - loss: 0.6352 - accuracy: 0.6819
Epoch 6/200
45/45 [==============================] - 0s 3ms/step - loss: 0.6324 - accuracy: 0.6819
Epoch 7/200
45/45 [==============================] - 0s 4ms/step - loss: 0.6298 - accuracy: 0.6819
Epoch 8/200
45/45 [==============================] - 0s 4ms/step - loss: 0.6276 - accuracy: 0.6819
Epoch 9/200
45/45 [==============================] - 0s 4ms/step - loss: 0.6256 - accuracy: 0.6819
Epoch 10/200
45/45 [==============================] - 0s 5ms/step - loss: 0.6240 - accuracy: 0.6819
Epoch 11/

45/45 [==============================] - 0s 3ms/step - loss: 0.5963 - accuracy: 0.6819
Epoch 164/200
45/45 [==============================] - 0s 3ms/step - loss: 0.5963 - accuracy: 0.6819
Epoch 165/200
45/45 [==============================] - 0s 5ms/step - loss: 0.5964 - accuracy: 0.6819
Epoch 166/200
45/45 [==============================] - 0s 3ms/step - loss: 0.5964 - accuracy: 0.6819
Epoch 167/200
45/45 [==============================] - 0s 4ms/step - loss: 0.5963 - accuracy: 0.6819
Epoch 168/200
45/45 [==============================] - 0s 3ms/step - loss: 0.5962 - accuracy: 0.6819: 0s - loss: 0.5992 - accuracy: 0.67
Epoch 169/200
45/45 [==============================] - 0s 3ms/step - loss: 0.5963 - accuracy: 0.6819
Epoch 170/200
45/45 [==============================] - 0s 2ms/step - loss: 0.5963 - accuracy: 0.6819
Epoch 171/200
45/45 [==============================] - 0s 3ms/step - loss: 0.5963 - accuracy: 0.6819
Epoch 172/200
45/45 [==============================] - 0s 3ms/step - 

# Prediction of testing data set

In [28]:
# Predict the classification of a new set of data
new_X_scaled = X_test_scaled
y_prediction=(nn_model.predict(new_X_scaled) > 0.5).astype("int32")


In [29]:
# Create a Dataframe for predictions
pred_df = pd.DataFrame(data=y_prediction, columns=["Prediction"])


In [30]:
# Create a Dataframe for y_test
single_results_df = pd.DataFrame({"Actual": y_test}).reset_index(drop=True)


In [31]:
# Create a Dataframe merging testing and prediction
neuronal_pred_df = pd.merge(single_results_df, pred_df,left_index=True, right_index=True)
neuronal_pred_df

,Actual,Prediction
0,1,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
476,0,0
477,0,0
478,1,0
479,1,0


In [32]:
#  Create a confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
matrix = confusion_matrix(y_test, y_prediction)
matrix_df = pd.DataFrame(
    matrix, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
matrix_df

,Predicted 0,Predicted 1
Actual 0,328,0
Actual 1,153,0


# Deep learning

In [33]:
# Define the model - deep neural net
number_input_features = len(X.columns)
hidden_nodes_layer1 = 4
hidden_nodes_layer2 = 4

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 4)                 32        
                                                                 
 dense_3 (Dense)             (None, 4)                 20        
                                                                 
 dense_4 (Dense)             (None, 1)                 5         
                                                                 
Total params: 57
Trainable params: 57
Non-trainable params: 0
_________________________________________________________________


In [34]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [35]:
fit_model = nn.fit(X_train,y_train,epochs=300)
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/300
45/45 [==============================] - 2s 6ms/step - loss: 9.0141 - accuracy: 0.3181
Epoch 2/300
45/45 [==============================] - ETA: 0s - loss: 4.8514 - accuracy: 0.31 - 0s 5ms/step - loss: 4.7371 - accuracy: 0.3181
Epoch 3/300
45/45 [==============================] - 0s 5ms/step - loss: 1.7900 - accuracy: 0.3201
Epoch 4/300
45/45 [==============================] - 0s 2ms/step - loss: 0.7291 - accuracy: 0.5882
Epoch 5/300
45/45 [==============================] - 0s 4ms/step - loss: 0.6912 - accuracy: 0.6653
Epoch 6/300
45/45 [==============================] - 0s 5ms/step - loss: 0.6851 - accuracy: 0.6708
Epoch 7/300
45/45 [==============================] - 0s 3ms/step - loss: 0.6798 - accuracy: 0.6722
Epoch 8/300
45/45 [==============================] - 0s 3ms/step - loss: 0.6751 - accuracy: 0.6750
Epoch 9/300
45/45 [==============================] - 0s 4ms/step - loss: 0.6709 - accuracy: 0.6736
Epoch 10/300
45/45 [==============================] - 0s 4ms/step -

45/45 [==============================] - 0s 4ms/step - loss: 0.5941 - accuracy: 0.7090
Epoch 163/300
45/45 [==============================] - 0s 4ms/step - loss: 0.5926 - accuracy: 0.7132
Epoch 164/300
45/45 [==============================] - 0s 4ms/step - loss: 0.5918 - accuracy: 0.7139
Epoch 165/300
45/45 [==============================] - 0s 3ms/step - loss: 0.5936 - accuracy: 0.7139
Epoch 166/300
45/45 [==============================] - 0s 3ms/step - loss: 0.5931 - accuracy: 0.7132
Epoch 167/300
45/45 [==============================] - 0s 4ms/step - loss: 0.5929 - accuracy: 0.7139
Epoch 168/300
45/45 [==============================] - 0s 4ms/step - loss: 0.5924 - accuracy: 0.7160
Epoch 169/300
45/45 [==============================] - 0s 3ms/step - loss: 0.5916 - accuracy: 0.7118
Epoch 170/300
45/45 [==============================] - 0s 3ms/step - loss: 0.5952 - accuracy: 0.7083
Epoch 171/300
45/45 [==============================] - 0s 4ms/step - loss: 0.5921 - accuracy: 0.7139
Epoc

In [36]:
# Predict the classification of a new set of data
new_X_scaled = X_test_scaled
y_prediction=(nn.predict(new_X_scaled) > 0.5).astype("int32")


In [37]:
# Create a Dataframe for predictions
pred_df = pd.DataFrame(data=y_prediction, columns=["Prediction"])
# Create a Dataframe for y_test
single_results_df = pd.DataFrame({"Actual": y_test}).reset_index(drop=True)
# Create a Dataframe merging testing and prediction
neuronal_pred_df = pd.merge(single_results_df, pred_df,left_index=True, right_index=True)
neuronal_pred_df

,Actual,Prediction
0,1,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
476,0,0
477,0,0
478,1,0
479,1,0


In [38]:
#  Create a confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
matrix = confusion_matrix(y_test, y_prediction)
matrix_df = pd.DataFrame(
    matrix, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
matrix_df

,Predicted 0,Predicted 1
Actual 0,328,0
Actual 1,151,2
